In [14]:
%load_ext autoreload
%autoreload 2

from gzip import GzipFile
import json
import os
import urllib
os.environ["TEST_SECRET"] = "setIt"
from app.main import app
from starlette.testclient import TestClient
from app.schemas import Collection, Document
import time



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
LABLAB_DATA_PATH = "../data/lablab_2023_06.json.gz"
with GzipFile(LABLAB_DATA_PATH) as data_file:
    lablab_data = json.load(data_file)
lablab_data[0]   

{'id': '4923644474f8835efa9c520bb58971746e27bdc6',
 'path': 'home/test',
 'locale': 'en',
 'title': 'Test',
 'description': '',
 'render': '<h1 class="toc-header" id="tets"><a href="#tets" class="toc-anchor">¶</a> Tets</h1>\n<video controls="true" id="presentationVideo">\n<source src="https://storage.googleapis.com/lablab-video-submissions/clect7lts0000356kwijsh28b%2Fraw%2Fsubmission-video-x-clect7lts0000356kwijsh28b-clf16992i03d72v6jdn152y2v.mp4" type="video/mp4">\n</video>\n<p></p>\n<p><span data-start="0" data-end="6.52">Hi everyone, today representing Logos, your personal knowledge base assistant. </span></p>\n<h2 class="toc-header" id="what-is-a-knowledge-base"><a href="#what-is-a-knowledge-base" class="toc-anchor">¶</a> What is a knowledge base?</h2>\n<p><span data-start="6.52" data-end="7.72">What\'s someone\'s knowledge base? </span><span data-start="7.72" data-end="12.44">It is often said that we are what we read, and this is true, but we are also what we </span><span data-sta

In [15]:
limit_docs = 100
batch = 10
recreate_index = True
collection_name = f'lablab{limit_docs}'
client = TestClient(app)
collections = client.get("api/v1/collections/").json() 
already_created = collection_name in collections
if already_created:
        print (f"already have {collection_name}")
if recreate_index or not already_created:
    if already_created:
        client.delete(f"api/v1/collections/{collection_name}/")
    print(f"upload {collection_name}")
    new_collection = Collection(name=collection_name, description=collection_name)
    assert client.post("api/v1/collections/", json=new_collection.dict()).status_code == 200
    for i in range(0,limit_docs,batch):
        assert client.post(f"api/v1/documents/{collection_name}/", json = lablab_data[i:min(i+batch,limit_docs)]).status_code == 200
        time.sleep(20)
        

already have lablab100
upload lablab100
[0.23413086, 0.35205078, -0.15881348]
[0.14538574, -0.13366699, -0.4831543]
[0.31030273, 0.011192322, 0.15478516]
[0.13134766, 0.2998047, -0.2619629]
lablab100:4923644474f8835efa9c520bb58971746e27bdc6:0
dict_keys(['path', 'doc_id', 'title', 'render_id', 'chunk_num', 'text', 'content', 'locale', 'vector'])
lablab100:4923644474f8835efa9c520bb58971746e27bdc6:1
dict_keys(['path', 'doc_id', 'title', 'render_id', 'chunk_num', 'text', 'content', 'locale', 'vector'])
lablab100:4923644474f8835efa9c520bb58971746e27bdc6:2
dict_keys(['path', 'doc_id', 'title', 'render_id', 'chunk_num', 'text', 'content', 'locale', 'vector'])
lablab100:4923644474f8835efa9c520bb58971746e27bdc6:3
dict_keys(['path', 'doc_id', 'title', 'render_id', 'chunk_num', 'text', 'content', 'locale', 'vector'])
[0.19665527, -0.08557129, -0.19030762]
[0.05291748, 0.03967285, -0.39501953]
lablab100:a84fc7be3c82d8d87369f2521dc7eb5d86c57fd6:0
dict_keys(['path', 'doc_id', 'title', 'render_id', '

In [17]:
response = client.get(f"api/v1/search/{collection_name}/?query=Questions about God")
result  = response.json()
result

Got questions!
Run search on: 1. What is the concept of God across different religions?
Run search on: 2. How is God commonly depicted or represented in various cultures?
Run search on: 3. What are some common attributes or characteristics associated with God in different belief systems?


{'qnas': {'0': {'question': '1. What is the concept of God across different religions?',
   'answer': 'The concept of God varies across different religions. In monotheistic religions like Christianity, Islam, and Judaism, God is understood as a singular, all-powerful deity who created and governs the universe [1]. In Hinduism, there are multiple gods and goddesses, and each represents a different aspect of the divine [1]. Buddhism, on the other hand, does not believe in a supreme deity, but instead focuses on spiritual enlightenment [1]. These are just a few examples, and there are many more variations in the concept of God across different religions.\n\nReferences:\n1. [1] home/test_what-is-a-knowledge-base. "What is a knowledge base?" \nURL: N/A\n"A knowledge base can help people integrate and vary all that information in more efficient ways. Have you ever thought to yourself something along those lines? If you ever had similar thoughts and needs, Logos is here to assist you."',
   '

In [19]:
print(json.dumps(result, indent=2))

{
  "qnas": {
    "0": {
      "question": "1. What is the concept of God across different religions?",
      "answer": "The concept of God varies across different religions. In monotheistic religions like Christianity, Islam, and Judaism, God is understood as a singular, all-powerful deity who created and governs the universe [1]. In Hinduism, there are multiple gods and goddesses, and each represents a different aspect of the divine [1]. Buddhism, on the other hand, does not believe in a supreme deity, but instead focuses on spiritual enlightenment [1]. These are just a few examples, and there are many more variations in the concept of God across different religions.\n\nReferences:\n1. [1] home/test_what-is-a-knowledge-base. \"What is a knowledge base?\" \nURL: N/A\n\"A knowledge base can help people integrate and vary all that information in more efficient ways. Have you ever thought to yourself something along those lines? If you ever had similar thoughts and needs, Logos is here t